In [48]:
import requests
from datetime import date, timedelta
import sqlite3
import math

In [49]:
def get_papers():
    today = date.today()
    yesterday = today - timedelta(days = 1)
    papers = requests.get("https://api.biorxiv.org/details/biorxiv/" + str(yesterday) + "/" + str(yesterday))
    papers_dic = papers.json()
    connection = sqlite3.connect('tweetbot.db')
    cursor = connection.cursor()
    cursor.execute('''DROP TABLE if exists yesterday_pubs''')
    cursor.execute('''CREATE TABLE 'yesterday_pubs'
                        ('doi', 'title','authors', 'author_corresponding', 
                      'author_corresponding_institution', 'date', 
                      'version', 'type','license','category','abstract','published','server')''')
    for child in papers_dic['collection']:
        cursor.execute('INSERT INTO yesterday_pubs VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?)', 
                      (child['doi'], child['title'], child['authors'], child['author_corresponding'], 
                      child['author_corresponding_institution'], child['date'], 
                      child['version'], child['type'], child['license'], 
                        child['category'],child['abstract'],child['published'],child['server']))
    connection.commit()
    pap = papers_dic['messages']
    print(pap)
    pepe = pap[0]
    for key,value in pepe.items():
        if key == 'total':
            n_papers =  value
    if n_papers > 100:
        total_loops = math.floor(n_papers/100)
        start = 101
        for n in range(total_loops):
            papers = requests.get("https://api.biorxiv.org/details/biorxiv/" + str(yesterday) + "/" + str(yesterday) + '/' + str(start))
            start += 100
            papers_dic = papers.json()
            connection = sqlite3.connect('tweetbot.db')
            cursor = connection.cursor()
            for child in papers_dic['collection']:
                cursor.execute('INSERT INTO yesterday_pubs VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?)', 
                              (child['doi'], child['title'], child['authors'], child['author_corresponding'], 
                              child['author_corresponding_institution'], child['date'], 
                              child['version'], child['type'], child['license'], 
                                child['category'],child['abstract'],child['published'],child['server']))
            connection.commit()
            start += 100
        print('ok')
    
get_papers()

[{'status': 'ok', 'interval': '2020-11-19:2020-11-19', 'cursor': 0, 'count': 100, 'count_new_papers': 75, 'total': 114}]
ok


In [50]:
def load_keywords():
    regex_list = []
    with open('search.txt') as f:
        lines = [i.strip() for i in f.readlines()]
        lowline = []
        for line in lines:
            if ') AND (' in line:
                prelowline = line.replace(') AND (', ' XXXX ')
                prelowline = prelowline.replace('(', '(abstract LIKE \'%').replace(')', '%\')').replace(' OR ', '%\' OR abstract LIKE \'%').replace(' AND ', '%\' AND abstract LIKE \'%')
                prelowline = prelowline.replace(' XXXX ', '%\') AND (abstract LIKE \'%')
                lowline.append(prelowline)
            elif ') OR (' in line:
                prelowline = line.replace(') OR (', ' XXXX ')
                prelowline = prelowline.replace('(', '(abstract LIKE \'%').replace(')', '%\')').replace(' OR ', '%\' OR abstract LIKE \'%').replace(' AND ', '%\' AND abstract LIKE \'%')
                prelowline = prelowline.replace(' XXXX ', '%\') OR (abstract LIKE \'%')
                lowline.append(prelowline)
            else:
                prelowline = line.replace(' OR ', '%\' OR abstract LIKE \'%').replace(' AND ', '%\' AND abstract LIKE \'%')
                prelowline = 'abstract LIKE \'%' + prelowline + '%\''
                lowline.append(prelowline)
        return lowline

load_keywords()


["(abstract LIKE '%pluripotent stem cells%' OR abstract LIKE '%embryonic stem cells%') AND (abstract LIKE '%cardiomyocytes%' OR abstract LIKE '%cardio*%' OR abstract LIKE '%cardiac differentiation%')",
 "(abstract LIKE '%pluripotent stem cells%' OR abstract LIKE '%embryonic stem cells%') AND (abstract LIKE '%microRNA%' OR abstract LIKE '%miRNA%' OR abstract LIKE '%lncRNA or lon-non-codingRNA or non-coding RNA%' OR abstract LIKE '%circRNA%')",
 "(abstract LIKE '%pluripotent stem cells%' OR abstract LIKE '%embryonic stem cells%') AND (abstract LIKE '%microfluidics%')",
 "(abstract LIKE '%pluripotent stem cells%' OR abstract LIKE '%embryonic stem cells%') AND (abstract LIKE '%gastrulation%' OR abstract LIKE '%tbx6%' OR abstract LIKE '%oct-6%' OR abstract LIKE '%oct6%')",
 "abstract LIKE '%C19MC%' OR abstract LIKE '%Chromosome 19 microRNA cluster%' OR abstract LIKE '%Chromosome 19 miRNA cluster%'",
 "abstract LIKE '%pluripotent stem cells%' OR abstract LIKE '%embryonic stem cell%'"]

In [51]:
def read_from_database():
    connection = sqlite3.connect('tweetbot.db')
    cursor = connection.cursor()
    keywords = load_keywords()
    for k in keywords:
        sql = "SELECT doi,title,version FROM yesterday_pubs WHERE " + k + 'COLLATE NOCASE'
        cursor.execute(sql)
        retrived = cursor.fetchall()
    return retrived
        
        
read_from_database()

[('10.1101/2020.11.19.389445',
  'Ngn2 induces diverse neuronal lineages from human pluripotency',
  '1'),
 ('10.1101/2020.11.17.386839',
  'Signatures Of TSPAN8 Variants Associated With Human Metabolic Regulation And Diseases',
  '1')]

In [52]:
def search_and_tweet():
    now = read_from_database()
    for line in now:
        doi = line[0]
        title = line[1]
        version = line[2]
        link = "https://www.biorxiv.org/content/" + doi +'v' + version
        print(title)
        n_char = len(title) + len(link)
        if n_char > 139:
            max_title_length = 136 - len(link)
            _title = title[:max_title_length]
            final_title = _title + '...'
        else:
            final_title = title
        print(final_title)
        print(link)
        print(len(final_title)+(len(link)))

search_and_tweet()

Ngn2 induces diverse neuronal lineages from human pluripotency
Ngn2 induces diverse neuronal lineages from human pluripotency
https://www.biorxiv.org/content/10.1101/2020.11.19.389445v1
121
Signatures Of TSPAN8 Variants Associated With Human Metabolic Regulation And Diseases
Signatures Of TSPAN8 Variants Associated With Human Metabolic Regulation And ...
https://www.biorxiv.org/content/10.1101/2020.11.17.386839v1
139


In [53]:
def tweet():
    now = read_from_database()
    for line in now:
        #for d in line:
        doi = line[0]
        title = line[1]
        version = line[2]
        link = "https://www.biorxiv.org/content/" + doi +'v' + version
        n_char = len(title) + len(link)
        if n_char > 139:
            max_title_length = 136 - len(link)
            _title = title[:max_title_length]
            final_title = _title + '...'
        else:
            final_title = title

        print(final_title)
        print("https://www.biorxiv.org/content/" + doi +'v' + version)
        
        
tweet()        

Ngn2 induces diverse neuronal lineages from human pluripotency
https://www.biorxiv.org/content/10.1101/2020.11.19.389445v1
Signatures Of TSPAN8 Variants Associated With Human Metabolic Regulation And ...
https://www.biorxiv.org/content/10.1101/2020.11.17.386839v1


In [65]:
import tweepy

def tweet_login():
    creds = []
    with open('credentials.txt') as f:
        creds = [i.strip() for i in f.readlines()]
        print(creds)
        auth = tweepy.OAuthHandler(creds[0], creds[1])
        print(auth)
        auth.set_access_token(creds[2], creds[3])
        api = tweepy.API(auth)
        try:
            api.verify_credentials()
            print("Authentication OK")
        except:
            print("Error during authentication")
        return api
                
tweet_login()

['1Wwp27ydWI2wEk7lZLQQ3h0RB', 'AHKe1QK4tGl0MLSYUz4LLEFkQxCrhwFabQIzcEmDEsUPT7fzk3', '1329555337427492864-t7JjutO5tC6Gdo2k0NSjFMNn2pIiWe', 'Uv31B52v6Fnf8BwhFmrzePxBejORIWKmk0HFwi9wA30tX']
Authentication OK


In [68]:
def tweet(title, link):
     api = tweet_login()
     api.update_status(title, link)

In [71]:
import time

In [77]:

def search_and_tweet():
    get_papers()
    now = read_from_database()
    api = tweet_login()
    for line in now:
        doi = line[0]
        print(doi)
        title = line[1]
        version = line[2]
        link = "https://www.biorxiv.org/content/" + doi +'v' + version
        n_char = len(title) + len(link)
        if n_char > 139:
            max_title_length = 106 - len(link)
            _title = title[:max_title_length]
            final_title = _title + '...'
        else:
            final_title = title
        with open('temp.txt', 'w') as f:
            f.write(final_title + '\n' + link)
        with open('temp.txt', 'r') as f:
            api.update_status(f.read())
            f.close()
        time.sleep(1)
        
search_and_tweet()

[{'status': 'ok', 'interval': '2020-11-19:2020-11-19', 'cursor': 0, 'count': 100, 'count_new_papers': 75, 'total': 114}]
ok
['1Wwp27ydWI2wEk7lZLQQ3h0RB', 'AHKe1QK4tGl0MLSYUz4LLEFkQxCrhwFabQIzcEmDEsUPT7fzk3', '1329555337427492864-t7JjutO5tC6Gdo2k0NSjFMNn2pIiWe', 'Uv31B52v6Fnf8BwhFmrzePxBejORIWKmk0HFwi9wA30tX']
Authentication OK
10.1101/2020.11.19.389445
10.1101/2020.11.17.386839
